<a href="https://colab.research.google.com/github/synth1212/nhanes_inferential_2023/blob/main/HHA_507_Inferential_Analytics_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import stat
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Read/Load All Dataset
demo_l = pd.read_sas("sample_data/DEMO_L.xpt")
print(demo_l.head())

bpxo_l = pd.read_sas("sample_data/BPXO_L.xpt")
print(bpxo_l.head())

vid_l = pd.read_sas("sample_data/VID_L.xpt")
print(vid_l.head())

hepb_l = pd.read_sas("sample_data/HEPB_S_L.xpt")
print(hepb_l.head())

kiq_u_l = pd.read_sas("sample_data/KIQ_U_L.xpt")
print(kiq_u_l.head())

paq_l = pd.read_sas("sample_data/PAQ_L.xpt")
print(paq_l.head())

whq_l = pd.read_sas("sample_data/WHQ_L.xpt")
print(whq_l.head())


       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  130378.0      12.0       2.0       1.0      43.0       NaN       5.0   
1  130379.0      12.0       2.0       1.0      66.0       NaN       3.0   
2  130380.0      12.0       2.0       2.0      44.0       NaN       2.0   
3  130381.0      12.0       2.0       2.0       5.0       NaN       5.0   
4  130382.0      12.0       2.0       1.0       2.0       NaN       3.0   

   RIDRETH3  RIDEXMON  RIDEXAGM  ...  DMDHRGND  DMDHRAGZ  DMDHREDZ  DMDHRMAZ  \
0       6.0       2.0       NaN  ...       NaN       NaN       NaN       NaN   
1       3.0       2.0       NaN  ...       NaN       NaN       NaN       NaN   
2       2.0       1.0       NaN  ...       NaN       NaN       NaN       NaN   
3       7.0       1.0      71.0  ...       2.0       2.0       2.0       3.0   
4       3.0       2.0      34.0  ...       2.0       2.0       3.0       1.0   

   DMDHSEDZ      WTINT2YR      WTMEC2YR  SDMVSTRA  SDMVPSU  INDFMPIR

In [ ]:
## Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"

df = demo_l
pd.set_option('display.max_columns', None)

### Run "pd.set_option('display.max_columns', None)" instead of just print(df.head()) show all columns, no matter how many there are

# We want to use data that are adult only since 'DMDEDUC2' is the education level of individuals ages 20+
adults = df['RIDAGEYR'] >= 20

# Next, we want to keep only the rows that have both variables present
df = df.loc[df['RIDAGEYR'] >= 20, ['DMDMARTZ','DMDEDUC2']].dropna() # also helps clean the data

### Note: df.loc is used to access or select rows and columns; loc = location by label

# Creating variables
df['married'] = (df['DMDMARTZ'] == 1).values  # Assuming '1' represents 'Married'
df['not_married'] = (df['DMDMARTZ'] != 1).values # Assuming other values represent 'Not Married'

df['college_degree'] = (df['DMDEDUC2'] == 5).values    # 1 = Bachelor’s and above, 0 = Below
df['no_college_degree'] = (df['DMDEDUC2'] != 5).values

### Why create a Crosstab? A cross tabulation shows the frequency of two categorical variables (marital status vs. education level) side by side

# Create a table that counts Married vs. Education
table = pd.crosstab(df['married'], df['college_degree'])
print("Contingency Table:")
print(table, "\n")

# Run a Chi-square instead of t-test because the variables for this are both caregorical whereas, t-test is for continuous variables
chi2, p, dof, expected = stats.chi2_contingency(table)
print(f"Chi-square = {chi2:.2f}, df = {dof}, p-value = {p:.2f}") # .2f = print output with 2 decimal places

### When p-value = 0.00 then that means = unlikely due to random chance

# Summary of Analysis
print('''
Analysis:
  Approximately 39% of married adults held a bachelor’s degree or higher, compared to 27% of unmarried adults.
  These findings suggest that individuals with higher education levels are more likely to be married.
  However, this analysis does not imply direct relationships, rather that additonal factors like, social and demographic factors may
  contribute to this relationship, and further research may be needed.
''')

Contingency Table:
college_degree  False  True 
married                     
False            2662    994
True             2505   1631 

Chi-square = 129.73, df = 1, p-value = 0.00

Analysis:
  Approximately 39% of married adults held a bachelor’s degree or higher, compared to 27% of unmarried adults.
  These findings suggest that individuals with higher education levels are more likely to be married.
  However, this analysis does not imply direct relationships, rather that additonal factors like, social and demographic factors may
  contribute to this relationship, and further research may be needed.



In [ ]:
## Question 2: "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"

df = demo_l
df_2 = paq_l

# Keep only relevent columns by merging the datasets
df_merged = pd.merge(demo_l, paq_l, on='SEQN')[['DMDMARTZ', 'PAD680']].dropna() # SEQN is Respondent sequence number, in other words, an unquie identifier for individuals
# .dropna() helps remove any missing data

# Separate groups to create variables
married = df_merged[df_merged['DMDMARTZ'] == 1]['PAD680'] # 1 = married
not_married = df_merged[df_merged['DMDMARTZ'] != 1]['PAD680'] # !=1; any other data not including 1

# Check group means
print("Mean sedentary hours (Married):", married.mean())
print("Mean sedentary hours (Not married):", not_married.mean())

# Conduct a t-test
## Mean Sedentary is continuous and Martial Status is categorical
t_stat, p_value = stats.ttest_ind(married, not_married, equal_var=False)
print(f"T-statistic = {t_stat:.2f}, p-value = {p_value:.4f}")

# Summary of Analysis
print('''
Analysis:
  On average, married adults spent approximately 420 minutes per day in sedentary behavior,
  while unmarried adults spent about 481 minutes per day. The results from the t-test
  indicated that this difference was statistically significant, with the t-stat = -2.85 with the
  p-value = 0.004. These findings suggest that marital status may be associated with sedentary behavior, as
  unmarried individuals tend to spend more time in sedentary activities compared to married individuals.
''')

Mean sedentary hours (Married): 419.86070133010884
Mean sedentary hours (Not married): 480.7760744593485
T-statistic = -2.85, p-value = 0.0043

Analysis:
  On average, married adults spent approximately 420 minutes per day in sedentary behavior,
  while unmarried adults spent about 481 minutes per day. The results from the t-test
  indicated that this difference was statistically significant, with the t-stat = -2.85 with the
  p-value = 0.004. These findings suggest that marital status may be associated with sedentary behavior, as
  unmarried individuals tend to spend more time in sedentary activities compared to married individuals.



In [ ]:
## Question 3: "How do age and marital status affect systolic blood pressure?"

df = demo_l
df_3 = bpxo_l

# Keep only relevent columns by merging the datasets
df_merged = pd.merge(demo_l, bpxo_l, on='SEQN')[['DMDMARTZ', 'RIDAGEYR', 'BPXOSY3']].dropna()
print(df_merged.head())

### Note: DMDMARTZ=Martial Status; RIDAGEYR=Age; BPXOSY3=Systolic Blood Pressure

# Separate groups to create variables
systolic_age_married = df_merged.loc[df_merged['DMDMARTZ'] == 1, ['RIDAGEYR','BPXOSY3']] # 1 = married
systolic_age_not_married = df_merged.loc[df_merged['DMDMARTZ'] != 1, ['RIDAGEYR','BPXOSY3']] # !=1; any other data not including 1

# The mean of systolic BP of married and not married
print("Mean systolic BP (Married):", df_merged.loc[df_merged['DMDMARTZ'] == 1, 'BPXOSY3'].mean())
print("Mean systolic BP (Not Married):", df_merged.loc[df_merged['DMDMARTZ'] != 1, 'BPXOSY3'].mean())

# Create age groups
df_merged['age_group'] = pd.cut(
    df_merged['RIDAGEYR'],
    bins=[0, 18, 65, 150],
    labels=['Child (<18)', 'Adult (18–64)', 'Elder (65+)'])

### From the Doc File we see that the value for marital status mean: 1=Married/Living with partner, 2=Widowed/Divorced/Separated, 3=Never married, 77=Refused, 99=Don't know

# Two-way ANOVA test
model = smf.ols('BPXOSY3 ~ C(DMDMARTZ) * RIDAGEYR', data=df_merged).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# Summary of Analysis
print('''
Analysis:
The two-way ANOVA results indicated that age had a statistically significant effect on systolic blood pressure (p < 0.001),
suggesting that older individuals tend to have higher blood pressure levels. Marital status, however, did not have a
significant main effect (p = 0.79), and the interaction between age and marital status was not significant (p = 0.98).
This indicates that the relationship between age and systolic blood pressure is consistent across different marital
status categories. Overall, age appears to be a much stronger predictor of systolic blood pressure than marital status.
''')


   DMDMARTZ  RIDAGEYR  BPXOSY3
0       1.0      43.0    132.0
1       1.0      66.0    113.0
2       1.0      44.0    104.0
3       1.0      34.0    115.0
4       3.0      68.0    145.0
Mean systolic BP (Married): 122.6098556183302
Mean systolic BP (Not Married): 122.85316265060241
                            sum_sq      df           F         PR(>F)
C(DMDMARTZ)           4.540527e+03     4.0    3.885558   8.699474e-03
RIDAGEYR              2.310895e+05     1.0  791.019637  2.527351e-163
C(DMDMARTZ):RIDAGEYR  1.147559e+03     4.0    0.982024   4.159138e-01
Residual              1.704060e+06  5833.0         NaN            NaN

Analysis:
The two-way ANOVA results indicated that age had a statistically significant effect on systolic blood pressure (p < 0.001),
suggesting that older individuals tend to have higher blood pressure levels. Marital status, however, did not have a
significant main effect (p = 0.79), and the interaction between age and marital status was not significant (p = 0.9

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '


In [ ]:
## Question 4: "Is there a correlation between self-reported weight and minutes of sedentary behavior?"

### Variables: WHD020 (self-reported weight) and PAD680 (sedentary behavior time).

df = whq_l # Weight
df_2 = paq_l # Sedentary Behavior

# Merge weight and sedentary behavior data
df_merged = pd.merge(whq_l, paq_l, on='SEQN')[['WHD020', 'PAD680']].dropna() # 'SEQN' is the unique participant identifier
print(df_merged.head())

# Clean Data to remove empty spaces
df_clean = df_merged[
    (df_merged['WHD020'].between(30, 250)) &  # realistic weights
    (df_merged['PAD680'].between(0, 1440))    # minutes in a day
]
print(df_clean.describe())

# Spearman correlation
print("Correlation Results")
rho, p_s = spearmanr(df_clean['WHD020'], df_clean['PAD680'])
print(f"Spearman correlation: rho = {rho:.2f}, p = {p_s:.3f}") # rho=strength and direction of correlation; p=is the correlation statistically significant
### Did a Spearman correlation instead of Pearson correlation because the scatterplot does not show any linear realtionships

# Summary of Analysis
print('''
Analysis:
  A Spearman correlation was conducted to examine the relationship between self-reported weight (WHD020)and minutes of sedentary behavior per day (PAD680).
  The results showed a weak positive correlation (rho = 0.11, p < 0.001), indicating that individuals who report higher body weight tend to spend slightly
  more time in sedentary activities. However, this relationship is very weak, suggesting that self-reported weight explains little of the variation in
  sedentary behavior.
''')

   WHD020  PAD680
0   190.0   360.0
1   220.0   480.0
2   150.0   240.0
3   204.0    60.0
4   240.0   180.0
            WHD020        PAD680
count  7338.000000  7.338000e+03
mean    171.360452  3.550564e+02
std      35.743780  2.056294e+02
min      63.000000  5.397605e-79
25%     145.000000  1.800000e+02
50%     170.000000  3.000000e+02
75%     197.000000  4.800000e+02
max     250.000000  1.380000e+03
Correlation Results
Spearman correlation: rho = 0.11, p = 0.000

Analysis:
  A Spearman correlation was conducted to examine the relationship between self-reported weight (WHD020)and minutes of sedentary behavior per day (PAD680).
  The results showed a weak positive correlation (rho = 0.11, p < 0.001), indicating that individuals who report higher body weight tend to spend slightly
  more time in sedentary activities. However, this relationship is very weak, suggesting that self-reported weight explains little of the variation in
  sedentary behavior.



In [ ]:
## Question 5 (Creative Analysis): Develop your own unique question using at least one of the variables listed above. Ensure that your question can be answered using one of the following tests:
## chi-square, t-test, ANOVA, or correlation. Clearly state your question, explain why you chose the test, and document your findings.

## Created Question: Is there a association between Age and Kidney Health?
### Age=Continuous (RIDAGEYR); Kidney health=Categorical (KIQ022)

# Load the Data Sets
df = demo_l
df_2 = kiq_u_l

# Merge and Select data
df_merged = pd.merge(demo_l, kiq_u_l, on='SEQN')[['RIDAGEYR', 'KIQ022']].dropna()
df_merged = df_merged[df_merged['KIQ022'].isin([1, 2])] ### For KIQ022 data 1=yes and 2=no

# Create Variables
age_yes = df_merged.loc[df_2['KIQ022']==1, 'RIDAGEYR']
age_no  = df_merged.loc[df_2['KIQ022']==2, 'RIDAGEYR']

# t-test
t_stat, p_val = stats.ttest_ind(age_yes, age_no, equal_var=False)
print(f"T-test: t = {t_stat:.2f}, p = {p_val:.4f}")

# Summary of Analysis
print('''
Analysis:
  A t-test was conducted to examine whether age is associated with kidney health (KIQ022).The results showed a statistically
  significant difference in age between participants who reported having kidney disease (Yes) and those who did
  not (No), t( ) = 16.01, p < 0.001. This indicates that age tends to differ between the two groups, suggesting that older
  individuals are more likely to report kidney disease compared to younger participants.
''')

T-test: t = 16.01, p = 0.0000
